In [51]:
import numpy as np
import pandas as pd
import random

### Death: Florida and its reference states

In [39]:
all_death = pd.read_csv('../20_intermediate_files/only_od_all_years.csv')
all_death.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,CTYNAME,STNAME
0,"Cochise County, AZ",4003.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,13.0,Cochise County,AZ
1,"Gila County, AZ",4007.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Gila County,AZ
2,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,411.0,Maricopa County,AZ
3,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Suicide (X60-X64),D2,118.0,Maricopa County,AZ
4,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,45.0,Maricopa County,AZ


In [40]:
fl_pop = pd.read_csv('../20_intermediate_files/fl_death_pop.csv')
fl_pop.STNAME.unique()

array(['Colorado', 'Florida', 'Louisiana', 'Nevada'], dtype=object)

In [46]:
# select only florida and its reference states
fl_ref_states = ['LA', 'NV', 'CO', 'FL']
fl_death = all_death[all_death['STNAME'].isin(fl_ref_states)]

# rename the abbrevation state name to its full name
fl_death_ = fl_death.replace({'NV': 'Nevada', 'FL': 'Florida', 'LA': 'Louisiana', 'CO': 'Colorado'})

# make sure the state names are the same
assert (fl_death_['STNAME'].unique() == fl_pop['STNAME'].unique()).all()

# rename year
fl_death_ = fl_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
fl_death_ = fl_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [38]:
# Collapsing the dataset to get total number of deaths per county per year
fl_agg_death = fl_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
fl_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,Colorado,Adams County,42.0
1,2003.0,Colorado,Arapahoe County,45.0
2,2003.0,Colorado,Boulder County,20.0
3,2003.0,Colorado,Denver County,107.0
4,2003.0,Colorado,El Paso County,53.0


In [47]:
# merge the two dataframes
fl_death_pop = pd.merge(fl_pop, fl_agg_death, how='left', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2000,350961,NaN,left_only
1,Colorado,Alamosa County,2000,14964,NaN,left_only
2,Colorado,Arapahoe County,2000,491701,NaN,left_only
3,Colorado,Archuleta County,2000,10038,NaN,left_only
4,Colorado,Baca County,2000,4501,NaN,left_only


In [48]:
# remove the year where we don't have the opoid death data
min_year = fl_death_['YEAR'].min()
max_year = fl_death_['YEAR'].max()

fl_death_pop = fl_death_pop[(fl_death_pop['YEAR'] >= min_year) & (fl_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
fl_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2003,378785,42.0,both
1,Colorado,Alamosa County,2003,15125,NaN,left_only
2,Colorado,Arapahoe County,2003,514406,45.0,both
3,Colorado,Archuleta County,2003,11167,NaN,left_only
4,Colorado,Baca County,2003,4106,NaN,left_only
...,...,...,...,...,...,...
2751,Nevada,Pershing County,2015,6620,NaN,left_only
2752,Nevada,Storey County,2015,3881,NaN,left_only
2753,Nevada,Washoe County,2015,442617,84.0,both
2754,Nevada,White Pine County,2015,9875,NaN,left_only


In [49]:
# missing values
fl_death_pop.isna().sum()

STNAME           0
CTYNAME          0
YEAR             0
POPULATION       0
Deaths        1933
_merge           0
dtype: int64

In [ ]:
# use random number to fill the missing values
fl_death_pop['Deaths'] = fl_death_pop['Deaths'].fillna(np.random.randint(0, 10, size=len(fl_death_pop)))

In [56]:
fl_death_pop['Deaths'].fillna(np.random.randint(0, 10))

0       42.0
1        4.0
2       45.0
3        4.0
4        4.0
        ... 
2751     4.0
2752     4.0
2753    84.0
2754     4.0
2755    24.0
Name: Deaths, Length: 2756, dtype: float64

### Death: Texas and its reference states

In [8]:
tx_pop = pd.read_csv('../20_intermediate_files/tx_death_pop.csv')
tx_pop.STNAME.unique()

array(['New York', 'Oregon', 'Texas', 'Wisconsin'], dtype=object)

In [9]:
# select only florida and its reference states
tx_ref_states = ['NY', 'OR', 'TX', 'WI']
tx_death = all_death[all_death['STNAME'].isin(tx_ref_states)]

# rename the abbrevation state name to its full name
tx_death_ = tx_death.replace({'NY': 'New York', 'OR': 'Oregon', 'TX': 'Texas', 'WI': 'Wisconsin'})

# make sure the state names are the same
assert (tx_death_['STNAME'].unique() == tx_pop['STNAME'].unique()).all()

# rename year
tx_death_ = tx_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
tx_death_ = tx_death_.drop(columns=['County', 'Year Code'])
tx_death_.head()

,County Code,YEAR,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,CTYNAME,STNAME
104,36001.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,13.0,Albany County,New York
105,36001.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,15.0,Albany County,New York
106,36005.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,157.0,Bronx County,New York
107,36005.0,2007.0,Drug poisonings (overdose) Suicide (X60-X64),D2,13.0,Bronx County,New York
108,36007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Broome County,New York


In [10]:
# merge the two dataframes
tx_death_pop = pd.merge(tx_pop, tx_death_, how='left', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,_merge
0,New York,Albany County,2000,294960,NaN,NaN,NaN,NaN,left_only
1,New York,Allegany County,2000,49916,NaN,NaN,NaN,NaN,left_only
2,New York,Bronx County,2000,1333854,NaN,NaN,NaN,NaN,left_only
3,New York,Broome County,2000,200319,NaN,NaN,NaN,NaN,left_only
4,New York,Cattaraugus County,2000,84017,NaN,NaN,NaN,NaN,left_only


In [12]:
# remove the year where we don't have the opoid death data
min_year = tx_death_['YEAR'].min()
max_year = tx_death_['YEAR'].max()

tx_death_pop = tx_death_pop[(tx_death_pop['YEAR'] >= min_year) & (tx_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
tx_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,_merge
0,New York,Albany County,2003,298491,NaN,NaN,NaN,NaN,left_only
1,New York,Allegany County,2003,50220,NaN,NaN,NaN,NaN,left_only
2,New York,Bronx County,2003,1353712,36005.0,Drug poisonings (overdose) Unintentional (X40-...,D1,131.0,both
3,New York,Broome County,2003,198364,NaN,NaN,NaN,NaN,left_only
4,New York,Cattaraugus County,2003,82968,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
5802,Wisconsin,Waukesha County,2015,396377,55133.0,Drug poisonings (overdose) Unintentional (X40-...,D1,34.0,both
5803,Wisconsin,Waupaca County,2015,51684,NaN,NaN,NaN,NaN,left_only
5804,Wisconsin,Waushara County,2015,23966,NaN,NaN,NaN,NaN,left_only
5805,Wisconsin,Winnebago County,2015,169383,55139.0,Drug poisonings (overdose) Unintentional (X40-...,D1,22.0,both


In [13]:
# missing values
tx_death_pop.isna().sum()

STNAME                                0
CTYNAME                               0
YEAR                                  0
POPULATION                            0
County Code                        4587
Drug/Alcohol Induced Cause         4587
Drug/Alcohol Induced Cause Code    4587
Deaths                             4587
_merge                                0
dtype: int64

### Death: Washington and its reference states

In [14]:
wa_pop = pd.read_csv('../20_intermediate_files/wa_death_pop.csv')
wa_pop.STNAME.unique()

array(['Hawaii', 'Oklahoma', 'Oregon', 'Washington'], dtype=object)

In [15]:
# select only florida and its reference states
wa_ref_states = ['HI', 'OK', 'OR', 'WA']
wa_death = all_death[all_death['STNAME'].isin(wa_ref_states)]

# rename the abbrevation state name to its full name
wa_death_ = wa_death.replace({'HI': 'Hawaii', 'OK': 'Oklahoma', 'OR': 'Oregon', 'WA': 'Washington'})

# make sure the state names are the same
assert (wa_death_['STNAME'].unique() == wa_pop['STNAME'].unique()).all()

# rename year
wa_death_ = wa_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
wa_death_ = wa_death_.drop(columns=['County', 'Year Code'])
wa_death_.head()

,County Code,YEAR,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,CTYNAME,STNAME
79,15001.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,14.0,Hawaii County,Hawaii
80,15003.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,75.0,Honolulu County,Hawaii
81,15003.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,14.0,Honolulu County,Hawaii
139,40017.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Canadian County,Oklahoma
140,40027.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,24.0,Cleveland County,Oklahoma


In [16]:
# merge the two dataframes
wa_death_pop = pd.merge(wa_pop, wa_death_, how='left', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,_merge
0,Hawaii,Hawaii County,2000,149095,NaN,NaN,NaN,NaN,left_only
1,Hawaii,Honolulu County,2000,875061,NaN,NaN,NaN,NaN,left_only
2,Hawaii,Kalawao County,2000,145,NaN,NaN,NaN,NaN,left_only
3,Hawaii,Kauai County,2000,58511,NaN,NaN,NaN,NaN,left_only
4,Hawaii,Maui County,2000,128754,NaN,NaN,NaN,NaN,left_only


In [17]:
# remove the year where we don't have the opoid death data
min_year = wa_death_['YEAR'].min()
max_year = wa_death_['YEAR'].max()

wa_death_pop = wa_death_pop[(wa_death_pop['YEAR'] >= min_year) & (wa_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
wa_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,County Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,_merge
0,Hawaii,Hawaii County,2003,156340,15001.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,both
1,Hawaii,Honolulu County,2003,888026,15003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,47.0,both
2,Hawaii,Honolulu County,2003,888026,15003.0,Drug poisonings (overdose) Suicide (X60-X64),D2,15.0,both
3,Hawaii,Kalawao County,2003,129,NaN,NaN,NaN,NaN,left_only
4,Hawaii,Kauai County,2003,60061,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
2200,Washington,Wahkiakum County,2015,3989,NaN,NaN,NaN,NaN,left_only
2201,Washington,Walla Walla County,2015,59970,NaN,NaN,NaN,NaN,left_only
2202,Washington,Whatcom County,2015,211942,53073.0,Drug poisonings (overdose) Unintentional (X40-...,D1,23.0,both
2203,Washington,Whitman County,2015,48224,NaN,NaN,NaN,NaN,left_only


In [18]:
# missing values
wa_death_pop.isna().sum()

STNAME                                0
CTYNAME                               0
YEAR                                  0
POPULATION                            0
County Code                        1571
Drug/Alcohol Induced Cause         1571
Drug/Alcohol Induced Cause Code    1571
Deaths                             1571
_merge                                0
dtype: int64